In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#구글드라이브연동

In [ ]:
import numpy as np
import cv2

In [ ]:
def set_region_of_interest(img, vertices):
    '''
    영상에서 관심 영역만 남김

    Parameters
    ----------
    img: array
        적용할 타겟 영상
    vertices: list of points
        관심 영역의 좌표점들이 저장된 리스트
    '''
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)

    return cv2.bitwise_and(img, mask)

def full_pipeline(img):
    '''
    차선 검출을 위한 전체 파이프라인 코드
    '''
    height, width = img.shape[:2]

    vertices = np.array([[(50,height),
                          (width/2-45, height/2+60),
                          (width/2+45, height/2+60),
                          (width-50,height)]],
                        dtype=np.int32)


    ## 1) 명암 영상에서 엣지 정보를 찾기 위해 영상의 색 공간 변환
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    ## 2) (optional) 영상 내의 노이즈를 감소시키기 위해 스무딩 효과 적용 (cv2.Canny()에 구현되어있지 않음)
    blur_img = cv2.GaussianBlur(gray_img, (3, 3), 0)

    ## 3) Canny edge detection을 사용하여 엣지 영상 검출
    edge_img = cv2.Canny(blur_img,
                         threshold1=70,   # gradient의 강도 < threshold1인 경우 무조건 엣지가 아니라고 판단
                         threshold2=175)  # gradient의 강도 > threshold2인 경우 무조건 엣지라고 판단(강한 엣지)
                                          # threshold1 < gradient의 강도 < threshold2인 경우 강한 엣지와 연결되어 있는 경우에만 엣지라고 판단

    ## 4) 타겟 차선 이외 부분에서의 엣지는 고려하지 않기 위해 영상에서 관심 영역만 남김
    roi_img = set_region_of_interest(edge_img, vertices)

    ## 5) Hough transform을 사용하여 조건을 만족하는 직선을 모두 검출
    lines = cv2.HoughLinesP(roi_img,
                            rho=1,            # rho값의 범위 (0~1 실수)
                            theta=np.pi/180,  # theta값의 범위(0~180 정수)
                            threshold=10,     # 변환 공간에서 만나는 점의 개수 기준
                            minLineLength=15, # 직선이 만족해야하는 최소 길이
                            maxLineGap=5)     # 얼마만큼 떨어져 있어도 하나의 직선으로 볼 것인지

    # ## 6) 찾은 직선을 결과 영상에 그리기
    # result = np.copy(img)
    # for line in lines:
    #     for x1,y1,x2,y2 in line:
    #         cv2.line(result, (x1, y1), (x2, y2), color=(0,0,255), thickness=5)

    ## 6) 찾은 직선을 결과 영상에 그리기
    result = np.copy(img)
    if lines is not None:  # lines가 None이 아닌 경우에만 반복
      for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(result, (x1, y1), (x2, y2), color=(0,0,255), thickness=5)

    return result

In [ ]:
def load_and_check_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"{image_path} 이미지를 불러오는 데 실패했습니다. 파일 경로를 확인해 주세요.")
    else:
        print(f"{image_path} 이미지가 성공적으로 불러와졌습니다.")
    return image

# 이미지 파일 경로
solidWhiteRight_path = '/content/drive/MyDrive/myproject/lane/solidWhiteRight.jpg'
solidYellowLeft_path = '/content/drive/MyDrive/myproject/lane/solidYellowLeft.jpg'

# 이미지 로드 및 확인
solidWhiteRight = load_and_check_image(solidWhiteRight_path)
solidYellowLeft = load_and_check_image(solidYellowLeft_path)

In [ ]:
from google.colab.patches import cv2_imshow

# 이미지 로드 및 확인
solidWhiteRight = load_and_check_image(solidWhiteRight_path)
solidYellowLeft = load_and_check_image(solidYellowLeft_path)

listOfFiles = [solidWhiteRight, solidYellowLeft]

for i, img in enumerate(listOfFiles):
    if img is None:
        continue  # 이미지가 None인 경우, 즉 로드에 실패한 경우 다음 이미지로 넘어감

    result = full_pipeline(img)
    cv2_imshow(result)  # cv2.imshow 대신 cv2_imshow 사용
    cv2.imwrite(f'lane_detection_{i}.jpg', result)  # 각 이미지에 대한 결과를 별도의 파일로 저장

cv2.destroyAllWindows()

In [ ]:
# 동영상 파일 열기
cap = cv2.VideoCapture('/content/drive/MyDrive/myproject/lane/lane.mp4')

# 원본 동영상의 프레임 레이트와 해상도 가져오기
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 결과 동영상을 저장하기 위한 준비
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, frame_rate, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 차선 검출
    result = full_pipeline(frame)

    # 결과 프레임을 동영상 파일에 쓰기
    out.write(result)

# 자원 해제
cap.release()
out.release()
cv2.destroyAllWindows()
